In [265]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import strptime
import datetime
import json
from pandas.io.json import json_normalize
from datetime import datetime

# Bitcoin data

In [391]:
bc_start = '20130428'
bc_end = datetime.today().strftime('%Y%m%d')
r = requests.get(f"https://coinmarketcap.com/currencies/bitcoin/historical-data/?start={bc_start}&end={bc_end}")
soup = BeautifulSoup(r.text, 'html.parser')
data = []
for row in soup.select('tr.text-right'):
    d = row.select('td')
    dt = d[0].get_text().replace(',', '').split(' ')
    date =  f"{dt[2]}-{strptime(dt[0],'%b').tm_mon}-{dt[1]}"
    data.append({'Date' : date, 'Open' : d[1].get_text(), 'High' : d[2].get_text(),
                 'Low' : d[3].get_text(), 'Close' : d[4].get_text(), 'Volume' : d[5].get_text(), 
                 'Market Cap' : d[6].get_text()}) 
df = pd.DataFrame.from_dict(data)
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Open'] = pd.to_numeric(df['Open'])
df['Open_g'] = 100 - ((100 * df['Open'].shift(-1)) / df['Open'])
df['Open_diff'] = -1 * (df['Open'].shift(-1) - df['Open'])

In [394]:
df.head()

,Close,Date,High,Low,Market Cap,Open,Volume,Open_g,Open_diff
0,4022.17,2019-03-24,4040.70,4006.19,"70,823,042,992",4035.16,"9,144,851,065",0.308538,12.45
1,4035.83,2019-03-23,4049.88,4015.96,"71,056,017,910",4022.71,"9,578,850,549",-0.144181,-5.80
2,4023.97,2019-03-22,4053.91,4021.54,"70,840,048,102",4028.51,"9,252,935,969",-1.376191,-55.44
3,4029.33,2019-03-21,4097.36,4005.15,"70,926,226,882",4083.95,"10,831,212,662",0.322237,13.16
4,4087.48,2019-03-20,4089.46,4031.11,"71,942,444,088",4070.79,"10,175,916,388",0.935936,38.10


In [392]:
cdf = df[df['Date'] >= '2019-03-01']
print(f"Last {cdf.shape[0]} days growth:", cdf['Open_diff'][:-1].sum(), '$')
print(f"Last {cdf.shape[0]} days day growth sum:", cdf['Open_g'][:-1].sum(), '%')
print(f"Last {cdf.shape[0]} days value growth:", 100 - (cdf['Open'].values[-1] * 100 / cdf['Open'][0]), '%')

Last 24 days growth: 181.39999999999964 $
Last 24 days day growth sum: 4.438965054063232 %
Last 24 days value growth: 4.495484689578603 %


# Bitcoin data by minutes
### url: http://bitcointicker.co/

In [381]:
dfm = pd.DataFrame()

In [397]:
print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
r = requests.get('https://bitcointicker.co/getHistCur.php?mins=60&cur=USD&exch=stamp')
print(r)
d = json.loads(r.text)
df = json_normalize(d)
df['price'] = pd.to_numeric(df['price'])
df = df.rename(index=str, columns={"time": "timestamp"})
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['date'] = df['timestamp'].map(lambda x : datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
df['time'] = df['timestamp'].map(lambda x : datetime.fromtimestamp(x).strftime('%H:%M:%S'))
df['price_gr'] = 100 - ((100 * df['price'].shift(1)) / df['price'])
df['price_diff'] = -1 * (df['price'].shift(1) - df['price'])
df['time_diff'] = df['timestamp'] - df['timestamp'].shift(1)
df = df.drop_duplicates(subset=['timestamp'], keep='last')
dfm = pd.concat([dfm, df]).drop_duplicates(subset=['timestamp'], keep='last')
print(dfm.shape, df.shape)

2019-03-25 07:25:57
<Response [200]>
(465, 8) (166, 8)


In [398]:
dfm.head()

,amount,price,timestamp,date,time,price_gr,price_diff,time_diff
0,0.0120,3957.73,1553461737,2019-03-24,23:08:57,NaN,NaN,NaN
1,0.0055,3957.73,1553461738,2019-03-24,23:08:58,0.0,-0.0,1.0
4,0.0220,3957.47,1553461743,2019-03-24,23:09:03,0.0,-0.0,0.0
5,0.0040,3957.47,1553461756,2019-03-24,23:09:16,0.0,-0.0,13.0
6,0.0040,3957.47,1553461769,2019-03-24,23:09:29,0.0,-0.0,13.0
